In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import hvplot.pandas

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ChrisClass")

from scipy.stats import linregress
from citipy import citipy

In [2]:
#Upload landfall and damage CSVs to notebook
landfall_df = pd.read_csv("Resources/landfall.csv")
damage_df = pd.read_csv("Resources/costliness.csv")

#Merge the two data frames into one
merged_df = pd.merge(landfall_df,damage_df,on = ['Season','Name'],how='outer')

#Drop rows with empty columns
merged_df = merged_df.dropna()

#Rename Season columns to Year
merged_df = merged_df.rename(columns={'Season':'Year'})

#Show dataframe preview
merged_df.head()


,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs
1,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B
5,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B
8,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B
15,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B
17,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B


In [3]:
#Create lists to store states, counties, and coordinates.
counties = []
states = []
lat_lngs = []

#Convert Lat and Long to a string
merged_df['Longitude'] = merged_df['Longitude'].astype(str)
merged_df['Latitude'] = merged_df['Latitude'].astype(str)

#Add lattitude and longitude values to list
lat_lngs = merged_df[['Latitude','Longitude']].values.tolist()

#Loop through lat_lng list to pull county and state names with geopy  
for lat_lng in lat_lngs:
    try:
        county = geolocator.reverse(lat_lng[0]+","+lat_lng[1])
        county = county.raw['address']['county']
        counties.append(county)
        state = geolocator.reverse(lat_lng[0]+","+lat_lng[1])
        state = state.raw['address']['state']
        states.append(state)
    #To handle errors if data isn't found
    except:
        counties.append('')
        states.append('')

#Add county and state names to dataframe
merged_df['County'] = counties
merged_df['State'] = states
merged_df['County'] = merged_df['County'].str.replace(' County','')
merged_df['County'] = merged_df['County'].str.replace(' Parish','')
merged_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
1,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston,Texas
5,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison,Mississippi
8,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary,Louisiana
15,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston,South Carolina
17,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South,Rhode Island


In [4]:
#Dictionary to store State names and abb.
state_to_abbrev = {"Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA",
    "Colorado": "CO","Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA",
    "Hawaii": "HI","Idaho": "ID","Illinois": "IL","Indiana": "IN","Iowa": "IA",
    "Kansas": "KS","Kentucky": "KY","Louisiana": "LA","Maine": "ME","Maryland": "MD",
    "Massachusetts": "MA","Michigan": "MI","Minnesota": "MN","Mississippi": "MS","Missouri": "MO",
    "Montana": "MT","Nebraska": "NE","Nevada": "NV","New Hampshire": "NH","New Jersey": "NJ",
    "New Mexico": "NM","New York": "NY","North Carolina": "NC","North Dakota": "ND","Ohio": "OH",
    "Oklahoma": "OK","Oregon": "OR","Pennsylvania": "PA","Rhode Island": "RI","South Carolina": "SC",
    "South Dakota": "SD","Tennessee": "TN","Texas": "TX","Utah": "UT","Vermont": "VT",
    "Virginia": "VA","Washington": "WA","West Virginia": "WV","Wisconsin": "WI","Wyoming": "WY","District of Columbia": "DC"}

#Replace State Names with codes
merged_df['State'] = merged_df['State'].replace(state_to_abbrev)
merged_df

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
1,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston,TX
5,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison,MS
8,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary,LA
15,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston,SC
17,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South,RI
18,4.0,8/24/92,1992,0905Z,25.5,-80.3,145.0,5.0,10,922,"CFL5,BFL4",Andrew,5.0,$60.2B,Miami-Dade,FL
19,4.0,8/26/92,1992,0830Z,29.6,-91.5,100.0,3.0,20,956,LA3,Andrew,5.0,$60.2B,St. Mary,LA
26,17.0,10/4/95,1995,2200Z,30.3,-87.1,100.0,3.0,50,942,"AFL3,IAL1",Opal,3.0,$9.6B,Escambia,FL
28,6.0,9/6/96,1996,0030Z,33.9,-78.0,100.0,3.0,40,954,NC3,Fran,3.0,$10.0B,Brunswick,NC
34,7.0,9/25/98,1998,1530Z,24.5,-81.8,90.0,2.0,25,981,BFL2,Georges,2.0,$11.5B,Monroe,FL


In [5]:
#Save new csv to avoid long run time of above code 
merged_df.to_csv('Resources/merged_df.csv',index=False)

In [6]:
#Upload new merged csv to notebook
hurricanes_df = pd.read_csv('Resources/merged_df.csv')
hurricanes_df

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston,TX
1,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison,MS
2,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary,LA
3,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston,SC
4,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South,RI
5,4.0,8/24/92,1992,0905Z,25.5,-80.3,145.0,5.0,10,922,"CFL5,BFL4",Andrew,5.0,$60.2B,Miami-Dade,FL
6,4.0,8/26/92,1992,0830Z,29.6,-91.5,100.0,3.0,20,956,LA3,Andrew,5.0,$60.2B,St. Mary,LA
7,17.0,10/4/95,1995,2200Z,30.3,-87.1,100.0,3.0,50,942,"AFL3,IAL1",Opal,3.0,$9.6B,Escambia,FL
8,6.0,9/6/96,1996,0030Z,33.9,-78.0,100.0,3.0,40,954,NC3,Fran,3.0,$10.0B,Brunswick,NC
9,7.0,9/25/98,1998,1530Z,24.5,-81.8,90.0,2.0,25,981,BFL2,Georges,2.0,$11.5B,Monroe,FL


In [7]:
#Upload HPI csv to dataframe
hpi_df = pd.read_csv('Resources/hpi_county.csv')
hpi_df.replace('.',"",inplace=True)
hpi_df.head()

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,AL,Autauga,1001,1986,,100.00,95.59,70.96
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24


In [8]:
#Create lists to store the year values of the years around a hurricane strike
previous_year = []
year_1 = []
year_2 = []
year_3 = []

#Create lists to store the Annual change values of the years around a hurricane strike
prev_ac = []
year_1_ac = []
year_2_ac = []
year_3_ac = []

#Create lists to store the HPI values of the years around a hurricane strike
prev_hpi = []
year_1_hpi = []
year_2_hpi = []
year_3_hpi = []

#Loop through the hpi data and pull the needed year values
for index in hpi_df.index:
    try:
        value = hpi_df.at[index-1,'Year']
        previous_year.append(value)
    except:
        previous_year.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+1,'Year']
        year_1.append(value)
    except:
        year_1.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+2,'Year']
        year_2.append(value)
    except:
        year_2.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+3,'Year']
        year_3.append(value)
    except:
        year_3.append('')

#Loop through the hpi data and pull the needed Annual Change values
for index in hpi_df.index:
    try:
        value = hpi_df.at[index-1,'Annual Change (%)']
        prev_ac.append(value)
    except:
        prev_ac.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+1,'Annual Change (%)']
        year_1_ac.append(value)
    except:
        year_1_ac.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+2,'Annual Change (%)']
        year_2_ac.append(value)
    except:
        year_2_ac.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+3,'Annual Change (%)']
        year_3_ac.append(value)
    except:
        year_3_ac.append('')

#Loop through the hpi data and pull the needed HPI values
for index in hpi_df.index:
    try:
        value = hpi_df.at[index-1,'HPI']
        prev_hpi.append(value)
    except:
        prev_hpi.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+1,'HPI']
        year_1_hpi.append(value)
    except:
        year_1_hpi.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+2,'HPI']
        year_2_hpi.append(value)
    except:
        year_2_hpi.append('')

for index in hpi_df.index:
    try:
        value = hpi_df.at[index+3,'HPI']
        year_3_hpi.append(value)
    except:
        year_3_hpi.append('')

#Create new columns and place the values on the data frame
hpi_df['Year -1'] = previous_year
hpi_df['Year +1'] = year_1
hpi_df['Year +2'] = year_2
hpi_df['Year +3'] = year_3
hpi_df['Annual Change (%) -1'] = prev_ac
hpi_df['Annual Change (%) +1'] = year_1_ac
hpi_df['Annual Change (%) +2'] = year_2_ac
hpi_df['Annual Change (%) +3'] = year_3_ac
hpi_df['HPI -1'] = prev_hpi
hpi_df['HPI +1'] = year_1_hpi
hpi_df['HPI +2'] = year_2_hpi
hpi_df['HPI +3'] = year_3_hpi
hpi_df.head()


,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,Year +2,Year +3,Annual Change (%) -1,Annual Change (%) +1,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3
0,AL,Autauga,1001,1986,,100.00,95.59,70.96,,1987,1988,1989,,-1.94,2.57,4.32,,98.06,100.58,104.92
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59,1986,1988,1989,1990,,2.57,4.32,-0.29,100.00,100.58,104.92,104.62
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38,1987,1989,1990,1991,-1.94,4.32,-0.29,2.52,98.06,104.92,104.62,107.25
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46,1988,1990,1991,1992,2.57,-0.29,2.52,3.09,100.58,104.62,107.25,110.57
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24,1989,1991,1992,1993,4.32,2.52,3.09,3.35,104.92,107.25,110.57,114.28


In [9]:
#Save Merged df to directory
hpi_df.to_csv('Output_CSVs/hpi_df.csv',index=False)

In [17]:
hpi_df_csv = pd.read_csv('Output_CSVs/hpi_df.csv')
hpi_df_csv

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,Year +2,Year +3,Annual Change (%) -1,Annual Change (%) +1,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3
0,AL,Autauga,1001,1986,NaN,100.00,95.59,70.96,NaN,1987.0,1988.0,1989.0,NaN,-1.94,2.57,4.32,NaN,98.06,100.58,104.92
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59,1986.0,1988.0,1989.0,1990.0,NaN,2.57,4.32,-0.29,100.00,100.58,104.92,104.62
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38,1987.0,1989.0,1990.0,1991.0,-1.94,4.32,-0.29,2.52,98.06,104.92,104.62,107.25
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46,1988.0,1990.0,1991.0,1992.0,2.57,-0.29,2.52,3.09,100.58,104.62,107.25,110.57
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24,1989.0,1991.0,1992.0,1993.0,4.32,2.52,3.09,3.35,104.92,107.25,110.57,114.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100427,WY,Weston,56045,2019,8.33,223.40,NaN,201.09,2018.0,2020.0,2021.0,2022.0,-0.90,4.36,4.93,6.87,206.21,233.13,244.62,261.41
100428,WY,Weston,56045,2020,4.36,233.13,NaN,209.86,2019.0,2021.0,2022.0,2023.0,8.33,4.93,6.87,-1.17,223.40,244.62,261.41,258.37
100429,WY,Weston,56045,2021,4.93,244.62,NaN,220.20,2020.0,2022.0,2023.0,NaN,4.36,6.87,-1.17,NaN,233.13,261.41,258.37,NaN
100430,WY,Weston,56045,2022,6.87,261.41,NaN,235.32,2021.0,2023.0,NaN,NaN,4.93,-1.17,NaN,NaN,244.62,258.37,NaN,NaN


In [22]:
#Create an aggregate table to determine mean and median by year by state and by nation
by_nation = hpi_df_csv.groupby(['Year'])
change_by_nation = by_nation['Annual Change (%)'].agg(['mean','median'])
by_state = hpi_df_csv.groupby(['State','Year'])
change_by_state = by_state['Annual Change (%)'].agg(['mean','median'])

#Add the state and national data to the HPI dataframe 
merged_df = pd.merge(hpi_df_csv,change_by_nation,on = ['Year'],how='outer')
merged_df2 = pd.merge(merged_df,change_by_state,on=['Year','State'],how='outer')
clean_new_hpi = merged_df2.rename(columns={'mean_x':'National AC Mean','median_x':'National AC Median',
                                          'mean_y':'State AC Mean','median_y':'State AC Median'})
#Display dataframe
clean_new_hpi


,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,...,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3,National AC Mean,National AC Median,State AC Mean,State AC Median
0,AK,Anchorage,2020,1975,NaN,100.00,56.05,38.81,2023.0,1976.0,...,9.52,8.93,288.26,109.25,119.65,130.33,NaN,NaN,NaN,NaN
1,AK,Fairbanks North Star,2090,1975,NaN,100.00,68.79,44.33,2023.0,1976.0,...,0.04,14.73,608.92,105.85,105.90,121.50,NaN,NaN,NaN,NaN
2,AL,Jefferson,1073,1975,NaN,100.00,47.95,32.53,2023.0,1976.0,...,9.94,11.49,330.46,108.90,119.73,133.49,NaN,NaN,NaN,NaN
3,AL,Lee,1081,1975,NaN,100.00,52.68,37.30,2023.0,1976.0,...,0.11,6.73,305.53,111.57,111.68,119.20,NaN,NaN,NaN,NaN
4,AL,Madison,1089,1975,NaN,100.00,39.81,33.06,2023.0,1976.0,...,11.16,5.39,169.70,110.27,122.58,129.18,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100427,WY,Sweetwater,56037,2023,6.56,673.62,343.66,237.06,2022.0,1986.0,...,-3.01,1.93,632.15,100.00,96.99,98.86,7.85995,7.63,4.909524,5.9
100428,WY,Teton,56039,2023,NaN,NaN,NaN,NaN,2022.0,1982.0,...,2.60,-1.63,1179.97,100.00,102.60,100.93,7.85995,7.63,4.909524,5.9
100429,WY,Uinta,56041,2023,2.77,361.18,521.19,297.72,2022.0,1992.0,...,7.61,8.63,351.46,100.00,107.61,116.89,7.85995,7.63,4.909524,5.9
100430,WY,Washakie,56043,2023,17.92,392.60,NaN,265.75,2022.0,1996.0,...,3.37,2.23,332.95,100.00,103.37,105.67,7.85995,7.63,4.909524,5.9


In [19]:
#Merge HPI data with hurricane data
hurricane_hpi_df = pd.merge(hurricanes_df,clean_new_hpi,on=['Year','County','State'],how='inner')
hurricane_hpi_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),...,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3,National AC Mean,National AC Median,State AC Mean,State AC Median
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,...,-4.40,-1.16,161.13,167.34,159.99,158.13,7.587261,5.725,9.237719,9.360
1,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,...,-5.08,-0.86,177.04,196.94,186.93,185.32,3.385471,3.365,6.701818,7.970
2,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,...,2.84,2.40,263.90,285.80,293.90,300.97,4.693742,4.200,3.666923,3.675
3,4.0,8/24/92,1992,0905Z,25.5,-80.3,145.0,5.0,10,922,...,4.36,4.63,239.18,261.13,272.52,285.14,2.964494,2.700,2.358750,2.060
4,17.0,10/4/95,1995,2200Z,30.3,-87.1,100.0,3.0,50,942,...,5.62,3.30,203.44,224.56,237.17,245.00,5.662343,5.540,3.318936,2.600


In [20]:
#Make a new dataframe with only relevant columns
clean_hurricane_hpi_df = hurricane_hpi_df[['Name','Max Winds (kt)','Adjusted Costs','County','State',
                        'Year -1','Year','Year +1','Year +2',
                        'Annual Change (%) -1','Annual Change (%)','Annual Change (%) +1','Annual Change (%) +2',
                        'HPI -1','HPI','HPI +1','HPI +2',
                        'National AC Mean','National AC Median','State AC Mean','State AC Median']]
clean_hurricane_hpi_df = clean_hurricane_hpi_df[(clean_hurricane_hpi_df['Year']<2022) & (clean_hurricane_hpi_df['Year']>1980)]
clean_hurricane_hpi_df

,Name,Max Winds (kt),Adjusted Costs,County,State,Year -1,Year,Year +1,Year +2,Annual Change (%) -1,...,Annual Change (%) +1,Annual Change (%) +2,HPI -1,HPI,HPI +1,HPI +2,National AC Mean,National AC Median,State AC Mean,State AC Median
0,Alicia,100.0,$9.4B,Galveston,TX,1982.0,1983,1984.0,1985.0,-2.03,...,1.64,-4.40,161.13,164.63,167.34,159.99,7.587261,5.725,9.237719,9.360
1,Elena,100.0,$3.8B,Harrison,MS,1984.0,1985,1986.0,1987.0,4.57,...,3.03,-5.08,177.04,191.15,196.94,186.93,3.385471,3.365,6.701818,7.970
2,Hugo,120.0,$22.7B,Charleston,SC,1988.0,1989,1990.0,1991.0,5.89,...,4.00,2.84,263.90,274.79,285.80,293.90,4.693742,4.200,3.666923,3.675
3,Andrew,145.0,$60.2B,Miami-Dade,FL,1991.0,1992,1993.0,1994.0,2.09,...,6.14,4.36,239.18,246.02,261.13,272.52,2.964494,2.700,2.358750,2.060
4,Opal,100.0,$9.6B,Escambia,FL,1994.0,1995,1996.0,1997.0,3.43,...,5.35,5.62,203.44,213.16,224.56,237.17,5.662343,5.540,3.318936,2.600
5,Fran,100.0,$10.0B,Brunswick,NC,1995.0,1996,1997.0,1998.0,8.11,...,8.86,7.20,192.77,203.54,221.57,237.51,3.964860,3.780,3.822892,4.040
6,Georges,90.0,$11.5B,Monroe,FL,1997.0,1998,1999.0,2000.0,6.20,...,5.86,14.89,292.04,313.55,331.92,381.35,3.897114,3.760,4.533208,3.970
7,Georges,90.0,$11.5B,Harrison,MS,1997.0,1998,1999.0,2000.0,4.08,...,5.74,5.58,243.21,256.11,270.82,285.95,3.897114,3.760,4.278913,4.050
8,Floyd,90.0,$12.1B,Brunswick,NC,1998.0,1999,2000.0,2001.0,7.20,...,8.12,5.06,237.51,252.09,272.55,286.34,3.715123,3.470,3.728211,3.770
9,Isabel,90.0,$9.3B,Carteret,NC,2002.0,2003,2004.0,2005.0,3.61,...,8.95,22.29,205.27,214.98,234.23,286.44,3.060076,2.570,2.179596,1.860


In [21]:
#Save hurricanes df to directory
hurricanes_df.to_csv('Output_CSVs/hurricanes_df.csv',index=False)

#Save Merged df to directory
clean_new_hpi.to_csv('Output_CSVs/clean_new_hpi.csv',index=False)

#Save clean df as CSV to directory
clean_hurricane_hpi_df.to_csv('Output_CSVs/clean_hurricane_hpi_data.csv',index=False)



hurricanes_df = Total Data for hurricane strikes in the US

hpi_df = Total HPI data with new columns for every county

clean_hurricane_hpi_df = Comined previous two data frames. All hurricane strikes with HPI data.

In [40]:
hurricanes_df

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State,City
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston,TX,hitchcock
4,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison,MS,long beach
7,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary,LA,patterson
14,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston,SC,mount pleasant
16,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South,RI,charlestown
17,4.0,8/24/92,1992,0905Z,25.5,-80.3,145.0,5.0,10,922,"CFL5,BFL4",Andrew,5.0,$60.2B,Miami-Dade,FL,lakes by the bay
18,4.0,8/26/92,1992,0830Z,29.6,-91.5,100.0,3.0,20,956,LA3,Andrew,5.0,$60.2B,St. Mary,LA,franklin
23,17.0,10/4/95,1995,2200Z,30.3,-87.1,100.0,3.0,50,942,"AFL3,IAL1",Opal,3.0,$9.6B,Escambia,FL,gulf breeze
25,6.0,9/6/96,1996,0030Z,33.9,-78.0,100.0,3.0,40,954,NC3,Fran,3.0,$10.0B,Brunswick,NC,long beach
31,7.0,9/25/98,1998,1530Z,24.5,-81.8,90.0,2.0,25,981,BFL2,Georges,2.0,$11.5B,Monroe,FL,key west


In [41]:
clean_new_hpi

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,...,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3,National AC Mean,National AC Median,State AC Mean,State AC Median
0,AK,Anchorage,2020,1975,NaN,100.00,56.05,38.81,2023.0,1976.0,...,9.52,8.93,288.26,109.25,119.65,130.33,NaN,NaN,NaN,NaN
1,AK,Fairbanks North Star,2090,1975,NaN,100.00,68.79,44.33,2023.0,1976.0,...,0.04,14.73,608.92,105.85,105.90,121.50,NaN,NaN,NaN,NaN
2,AL,Jefferson,1073,1975,NaN,100.00,47.95,32.53,2023.0,1976.0,...,9.94,11.49,330.46,108.90,119.73,133.49,NaN,NaN,NaN,NaN
3,AL,Lee,1081,1975,NaN,100.00,52.68,37.30,2023.0,1976.0,...,0.11,6.73,305.53,111.57,111.68,119.20,NaN,NaN,NaN,NaN
4,AL,Madison,1089,1975,NaN,100.00,39.81,33.06,2023.0,1976.0,...,11.16,5.39,169.70,110.27,122.58,129.18,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100427,WY,Sweetwater,56037,2023,6.56,673.62,343.66,237.06,2022.0,1986.0,...,-3.01,1.93,632.15,100.00,96.99,98.86,7.85995,7.63,4.909524,5.9
100428,WY,Teton,56039,2023,NaN,NaN,NaN,NaN,2022.0,1982.0,...,2.60,-1.63,1179.97,100.00,102.60,100.93,7.85995,7.63,4.909524,5.9
100429,WY,Uinta,56041,2023,2.77,361.18,521.19,297.72,2022.0,1992.0,...,7.61,8.63,351.46,100.00,107.61,116.89,7.85995,7.63,4.909524,5.9
100430,WY,Washakie,56043,2023,17.92,392.60,NaN,265.75,2022.0,1996.0,...,3.37,2.23,332.95,100.00,103.37,105.67,7.85995,7.63,4.909524,5.9


In [42]:
clean_hurricane_hpi_df

,Name,Max Winds (kt),Adjusted Costs,County,State,Year -1,Year,Year +1,Year +2,Annual Change (%) -1,...,Annual Change (%) +1,Annual Change (%) +2,HPI -1,HPI,HPI +1,HPI +2,National AC Mean,National AC Median,State AC Mean,State AC Median
0,Alicia,100.0,$9.4B,Galveston,TX,1982.0,1983,1984.0,1985.0,-2.03,...,1.64,-4.40,161.13,164.63,167.34,159.99,7.587261,5.725,9.237719,9.360
1,Elena,100.0,$3.8B,Harrison,MS,1984.0,1985,1986.0,1987.0,4.57,...,3.03,-5.08,177.04,191.15,196.94,186.93,3.385471,3.365,6.701818,7.970
2,Hugo,120.0,$22.7B,Charleston,SC,1988.0,1989,1990.0,1991.0,5.89,...,4.00,2.84,263.90,274.79,285.80,293.90,4.693742,4.200,3.666923,3.675
3,Andrew,145.0,$60.2B,Miami-Dade,FL,1991.0,1992,1993.0,1994.0,2.09,...,6.14,4.36,239.18,246.02,261.13,272.52,2.964494,2.700,2.358750,2.060
4,Opal,100.0,$9.6B,Escambia,FL,1994.0,1995,1996.0,1997.0,3.43,...,5.35,5.62,203.44,213.16,224.56,237.17,5.662343,5.540,3.318936,2.600
5,Fran,100.0,$10.0B,Brunswick,NC,1995.0,1996,1997.0,1998.0,8.11,...,8.86,7.20,192.77,203.54,221.57,237.51,3.964860,3.780,3.822892,4.040
6,Georges,90.0,$11.5B,Monroe,FL,1997.0,1998,1999.0,2000.0,6.20,...,5.86,14.89,292.04,313.55,331.92,381.35,3.897114,3.760,4.533208,3.970
7,Georges,90.0,$11.5B,Harrison,MS,1997.0,1998,1999.0,2000.0,4.08,...,5.74,5.58,243.21,256.11,270.82,285.95,3.897114,3.760,4.278913,4.050
8,Floyd,90.0,$12.1B,Brunswick,NC,1998.0,1999,2000.0,2001.0,7.20,...,8.12,5.06,237.51,252.09,272.55,286.34,3.715123,3.470,3.728211,3.770
9,Isabel,90.0,$9.3B,Carteret,NC,2002.0,2003,2004.0,2005.0,3.61,...,8.95,22.29,205.27,214.98,234.23,286.44,3.060076,2.570,2.179596,1.860
